In [19]:
import pandas as pd
import numpy as np
from groq import Groq
from openai import OpenAI
from pyairtable import Table,Api
from pprint import pprint
import networkx as nx
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.cluster.util import cosine_distance


In [20]:
api = Api('patkECjZGaeJ3Rj2F.902a4dc39c614cbcba594a1d909db35a39dcd69f43d2bf692aaf32268c4d60d7')
tableMessages = api.table('appY85YsYveVhA1eC', 'Messages')
data=tableMessages.all()
fields = [record['fields'] for record in data]
df=pd.DataFrame(fields)

In [21]:
dff = df['other'].dropna().to_list()

In [22]:
print(dff)

['non ci interessa lavorare specificatamente sulle loro tecnologie', 'non chiederei se sta bene nel primo messaggio, probabilmente troppo lungo', "completamente errato, anche il nome dell'azienda errato: Airmbag Studio", 'saluti saluti al fondo non va bene, troppo lungo', 'piccola nota, sarebbe meglio usare digital strategy e non strategies (in italiano si usa il singolare se non sbaglio)', "non ha senso parlare di impegno nel mondo tecnologico di un'azienda che lavora unicamente nel mondo tecnologico", "non parla dell'azienda che è da interpellare", 'troppo lungo', "Airbag Studio è un nome, non va bene l'articolo determinativo all'inizio", "è un'azienda 100% tecnologica, non possiamo portarli nel mondo della tecnologia", 'sarei onorato non va bene, toglierei spa', 'diversi errori dovuti probabilmente alal traduzione, piccolezza: dopo la virgola parte con la maiuscola.', "suona un po' fake, troppo lungo", 'ripete digital, messaggio da affinare', "mi presente come è un po' strano, in ge

In [ ]:
pprint(dff)

In [ ]:
%pip install transformers sentence-transformers

In [10]:
from networkx.algorithms.link_analysis.pagerank_alg import pagerank
from sentence_transformers import SentenceTransformer, util

/Users/neralbcika/anaconda3/envs/stage-ai/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [11]:
class BERTSummarizer:

    def __init__(self, pretrained_model='all-MiniLM-L6-v2'):
        self.bert_model = SentenceTransformer(pretrained_model)

    def create_graph(self, sentences, sentence_vectors):
        G = nx.Graph()
        for i, s1 in enumerate(sentences):
            for j, s2 in enumerate(sentences):
                # do not compute similarity with itself
                if s1 != s2:
                    similarity = util.pytorch_cos_sim(sentence_vectors[i], sentence_vectors[j])
                    G.add_edge(i, j, weight=similarity)
        return G

    def summarize(self, sentences, N=2, return_list=False):
        sentence_vectors = self.bert_model.encode(sentences)
        G = self.create_graph(sentences, sentence_vectors)
        try:
            pr_scores = pagerank(G, max_iter=1000)
        except Exception as e:
            print("The PageRank algorithm failed to converge. Returning the top sentences according to their position in the text.")
            pr_scores = {i: N-i for i in range(len(sentences))}

        ordered_pr = dict(sorted(pr_scores.items(), key=lambda item: item[1], reverse=True))
        ordered_keys = list(ordered_pr.keys())
        summary_sents = []
        for ind in ordered_keys[:N]:
            summary_sents.append(sentences[ind])
        if return_list:
            return summary_sents
        else:
            return " ".join(summary_sents)

In [17]:
bs = BERTSummarizer()
summary = bs.summarize(dff)
print (summary)

The PageRank algorithm failed to converge. Returning the top sentences according to their position in the text.
non ci interessa lavorare specificatamente sulle loro tecnologie non chiederei se sta bene nel primo messaggio, probabilmente troppo lungo


In [25]:
phrases="\n".join(dff)
num_sent=4
model = "llama3-70b-8192"
client = Groq(
        api_key='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',
    )

context = f"""You are given a list of phrases in Italian. Your task is to understand the semantic meaning of these phrases and generate sentences that best describe the essence of the phrases. Please generate {num_sent} descriptive sentences. Here are the phrases:\n\n{phrases}"""

messages = [
    {"role": "system", "content": "You are an AI language model."},
    {"role": "user", "content": context}
]

chat_completion = client.chat.completions.create(
                    messages=messages,
                    model=model,
                    max_tokens=32600,
                    temperature=1
                )
llm_response = chat_completion.choices[0].message.content
print(llm_response)


SyntaxError: invalid syntax. Perhaps you forgot a comma? (906901674.py, line 16)